# 文件读写

## 读文件

在磁盘上读写文件的功能是由操作系统提供的，现代操作系统不允许普通程序直接操作磁盘，  
所以读写文件就是请求操作系统打开一个文件对象，然后通过操作系统提供的接口，  
从这个文件对象中读取数据（读文件），或者把数据写入这个文件对象（写文件）。

In [ ]:
# 要以读文件的模式打开一个文件对象，可使用open()传入文件名和标示符，标示符'r'表示读
# 如果文件不存在，open()就会抛出一个IOError的错误，并且给出错误码和详细的信息
f = open('/Users/test.txt', 'r')

In [ ]:
# 打开非UTF-8编码的文本文件，要给open()传入encoding参数，例如GBK编码的文件
f = open('/Users/gbktest.txt', 'r', encoding='gbk')

对于编码不规范的文件，可能会遇到`UnicodeDecodeError`，因为在文本文件中可能夹杂了一些非法编码的字符。  
针对这种情况，open()还接收一个`errors`参数，表示如遇编码错误如何处理，其中最简单的方式是直接忽略：

In [ ]:
f = open('/Users/errorstest.txt', 'r', encoding='gbk', errors='ignore')

In [ ]:
# 前面讲的都是读取文本文件，要读取二进制文件，比如图片、视频，用'rb'标示符打开文件即可
f = open('/Users/test.png', 'rb')

In [ ]:
# 文件打开成功后调用read()，可以读取文件的全部内容，Python会把内容读到内存，用一个str对象表示
f.read()

In [ ]:
# 最后要用close()关闭文件，因为文件对象会占用操作系统的资源
f.close()

In [ ]:
# 文件读写时都有可能产生IOError，导致后面的close()不会调用
# 为了保证无论如何都能正确地关闭文件，可以使用try...finally
try:
    f = open('/path/to/file', 'r')
    print(f.read())
finally:
    if f:
    f.close()

In [ ]:
# with语句可以自动调用close()，效果和try...finally一样，但是代码更简洁
with open('/path/to/file', 'r') as f:
    print(f.read())

调用`read()`会一次性读取文件的全部内容，如果文件很大，内存就爆了，  
出于保险起见可以反复调用`read(size)`，每次最多读取size个字节的内容。  
另外，调用readline()可以一次性读取所有内容并按行返回list，常用于读取配置文件。

有read()方法的对象，比如open()返回的对象，统称为`file-like Object`。
这种对象不要求从特定类继承，除`file`外，还可以是内存的字节流、网络流、自定义流等，
比如`StringIO`和`BytesIO`，就是在内存中创建的file-like Object。

## 写文件

写文件和读文件几乎是一样的，区别是调用open()时，传入标识符'w'或'wb'表示写文本文件或写二进制文件。    
可以反复调用`write()`来写入文件，但是务必要调用close()来关闭文件。  
因为操作系统往往不会立刻把数据写入磁盘，而是放到内存缓存起来，空闲时再慢慢写入。  
只有调用close()时，操作系统才保证把没有写入的数据全部写入磁盘，忘记调用close()可能造成数据丢失。

In [ ]:
# 和读文件一样，还是使用with语句简洁又安全
with open('/Users/test.txt', 'w') as f:
    f.write('Hello, world!')

要写入特定编码的文本文件，请给open()传入encoding参数，将字符串自动转换成指定编码。

## StringIO和BytesIO

对磁盘的读写操作速度没有直接对内存操作快，这里介绍对内存操作的文件对象StringIo和BytesIO。

In [ ]:
# 先创建一个StringIO对象，然后调用write()写入str（字符串必须为unicode编码）
# getvalue()方法用于获得写入后的str，注意最后不要忘记调用close()
from io import StringIO
f = StringIO()
f.write(u'hello')  
print(f.getvalue())
f.close()          

In [ ]:
# 如果要操作二进制数据，就需要使用BytesIO
from io import BytesIO
fother = open('/Users/test.png','rb')  
data = fother.read()
fother.close()
# data是字节类型，getvalue()返回的结果也是字节类型
f = BytesIO(data)    
print(f.getvalue())  
f.close()